**NOTE** `phimats` environment should be used as kernel

In [5]:
import h5py

import PreProcessing
from PreProcessing import PreProcessing as PP, ReadMesh

import BoundaryConditions
from BoundaryConditions import *

import PostProcessing
from PostProcessing import WriteXDMF

### Simulation data

In [ ]:
# Simulation name
Simul = "FluxGB"
# Simulation type
SimulType = "GBTrapping"
# Element name
elementName = "triangle"  
# Element sets
nElementSets = 1
# Number of steps to apply the load
nSteps = 100

### Read mesh file 

The mesh file is hosted on Zenodo due to its large size.  
To download it, paste the command below into your terminal.  
**Note:** Make sure you're in the desired working directory.

```bash
wget https://zenodo.org/records/15380594/files/RVE_mesh.inp
```

In [4]:
modelName = "RVE_mesh"

mesh = ReadMesh(modelName)
print(mesh)

<meshio mesh object>
  Number of points: 1000000
  Number of cells:
    triangle: 1996002
  Point sets: plate
  Cell sets: plate


### Element data

In [15]:
elementName = "triangle" 	# meshio compatible element name
nNodes = mesh.points.shape[0]            # Total number of nodes
nodCoord = mesh.points[:,0:2]            # Node coordinates
nElements = mesh.cells_dict[elementName].data.shape[0]  # Total number of elements
elemNodeConn = mesh.cells_dict[elementName]        # Node connectivity for all elements

### Read phase-field data $g(\phi)$

In [16]:
# gPhi mapped to finite element mesh

fh5 = h5py.File("RVE_GB.hdf5", "r")

try:
	gPhi = fh5["gPhi"][()]    
	fh5.close()
except:
    fh5.close()


## Diffusion and trapping data

In [17]:
# Diffusivity data
T = 300                    # Temperature [K]
R = 8.31446261815324       # Universal gas constant [J/mol.K]
dt = 10e-3  			   # Time increment [s]
N = 6          	# Number of interstitial sites per reference lattice atom [tetrahedral sites in BCC]

D01 = 8.45e-8; DQ1 = 5000  # Reference lattice diffusivity m²/s
DL = D01*np.exp(-DQ1/(R*T))

D02 = 8.45e-8; DQ2 = 5000  # GB diffusivity m²/s
DT = D02*np.exp(-DQ2/(R*T))

HGB = 10000 # GB enrichment enthalpy[J/mol]

Vm = 7.09e-6  # Molar volume of Fe [m³/mol]
Vgb = 2*Vm    # Molar volume around GBs  [m³/mol]

kappa_GB = 4*N*HGB/Vm # GB occupancy enrichment ratio
KGB = np.exp(0.25*kappa_GB*Vm/(N*R*T))
print("KGB: ", KGB)

theta_b = 3.453e-3*Vm/N
print(theta_b)

cL = theta_b*N/Vm  # Concentration  [mol/m³]
print("cL", cL)

theta_gb = theta_b*KGB # Occupancy at the grain boundaries

c_GB = theta_gb/Vgb
print("c_GB", c_GB)
Z_gb = c_GB/cL # Concentration enrichment ratio
print("Z_gb", Z_gb)
zeta_gb = 4*R*T*np.log(Z_gb)
print("zeta_gb", zeta_gb)

print("Eq Con GB: ", np.exp(0.25*zeta_gb/(R*T)))
print("Eq Con TJ: ", np.exp(0.33*zeta_gb/(R*T)))

KGB:  55.09607620783739
4.080295e-09
cL 0.003453
c_GB 0.01585389592880521
Z_gb 4.591339683986449
zeta_gb 15207.20386088472
Eq Con GB:  4.591339683986449
Eq Con TJ:  7.477562307558887


In [18]:
# Material parameters dict

Materials = {
    "Material_1" : {
	"D0x1" : D01,
 	"D0y1" : D01,
	"DQx1" : DQ1,
	"DQy1" : DQ1,
 	"D0x2" : D02,
 	"D0y2" : D02,
	"DQx2" : DQ2,
	"DQy2" : DQ2,
 	"zeta_GB"  : zeta_gb,
	}
}

Materials

{'Material_1': {'D0x1': 8.45e-08,
  'D0y1': 8.45e-08,
  'DQx1': 5000,
  'DQy1': 5000,
  'D0x2': 8.45e-08,
  'D0y2': 8.45e-08,
  'DQx2': 5000,
  'DQy2': 5000,
  'zeta_GB': np.float64(15207.20386088472)}}

### Apply boundary conditions

In [19]:
lx = 100e-6    # width of the RVE
ly = 100e-6    # height of the RVE

exitNods = []

# Set keyworkd argument `Charging` to true for charging simulation
conBCs, exitNods = PermeationX(ly, cL, mesh, Charging=False)

# Equilibrium boundary conditions
for i in range(len(conBCs)):
    conBCs[i][1] = conBCs[i][1]*np.exp(gPhi[conBCs[i][0]]*zeta_gb/(R*T))
    
# Write boundary conditions in a .vtu file for checking in Paraview. recommended view is `Points`. 
BCmesh = WriteConBCs(Simul, elementName, mesh, conBCs)

### Initialize simulation object

In [20]:
reload(PreProcessing)
from PreProcessing import PreProcessing as PP

In [21]:
SimulationData = {
    "Simul"            : Simul,
    "SimulType"		   : SimulType,
    "mesh"             : mesh,
    "gPhi"             : gPhi,
    "elementName"      : elementName,
    "nElementSets"     : nElementSets,
    "presBCs"          : conBCs,
    "conB"             : cL,
    "exitNods"         : exitNods,
    "dt"               : dt,
    "nSteps"           : nSteps,
    "Materials"        : Materials,
    "T"                : T,
}

# Preprocessing object
GB_Sim = PP(SimulationData)

# Create the input file Simul+"_in.hdf5" file
GB_Sim.WriteInputFile()

HDF5 file 'Permeation_Tri3_in.hdf5' opened successfully in write mode.
HDF5 file closed successfully.


In [22]:
# Create the output file Simul+"_out.hdf5" file
GB_Sim.WriteOutputFile(OVERWRITE=True, FLUX=True, AVFLUX=True)

### Generate xdmf visualization file

In [24]:
WriteXDMF(Simul, elementName, nSteps+1, "Transport2D", FLUX=True)